In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import cv2 as cv
import os
import time
import pandas as pd
import matplotlib.pyplot as plt

from skimage.feature import hog
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import (StratifiedKFold, StratifiedShuffleSplit, 
                                     train_test_split, GridSearchCV, cross_val_score, 
                                     RandomizedSearchCV)

In [ ]:
!mkdir training_data_64
!mkdir training_data_64/cheetah
!mkdir training_data_64/jaguar
!mkdir training_data_64/tiger

In [ ]:
!unrar e "/content/drive/MyDrive/Dataset/training_data_64/cheetah.rar" "/content/training_data_64/cheetah"
!unrar e "/content/drive/MyDrive/Dataset/training_data_64/jaguar.rar" "/content/training_data_64/jaguar"
!unrar e "/content/drive/MyDrive/Dataset/training_data_64/tiger.rar" "/content/training_data_64/tiger"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/Dataset/training_data_64/cheetah.rar

Extracting  /content/training_data_64/cheetah/aug_0_6121912.jpg            0%  OK 
Extracting  /content/training_data_64/cheetah/aug_100_7164689.jpg          1%  OK 
Extracting  /content/training_data_64/cheetah/aug_101_2901784.jpg          1%  OK 
Extracting  /content/training_data_64/cheetah/aug_102_9506627.jpg          2%  OK 
Extracting  /content/training_data_64/cheetah/aug_103_3202380.jpg          2%  OK 
Extracting  /content/training_data_64/cheetah/aug_104_9121367.jpg          3%  OK 
Extracting  /content/training_data_64/cheetah/aug_105_6957174.jpg          3%  OK 
Extracting  /content/training_data_64/cheetah/aug_106_9084765.jpg          4%  OK 
Extracting  /content/training_data_64/cheetah/aug_107_6815822.jpg          4%  OK 
Extracting  /content/train

In [ ]:
def create_dataset(folders):
    image_data = []
    label_data = []
    IMG_SIZE = 64
    CATEGORIES = ['cheetah', 'jaguar', 'tiger']
    
    for category in CATEGORIES:
        class_name = CATEGORIES.index(category)
        
        for file in os.listdir(os.path.join(folders, category)):
            path = os.path.join(folders, category, file)
            image = cv.imread(path, 0)
            #image = cv.cvtColor(image, cv.COLOR_BGR2RGB)    # convert BGR to RGB
            #norm_image = cv.normalize(image, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_64F)    # normalization
            
            # CLAHE (Contrast Limited Adaptive Histogram Equalization)
            clahe = cv.createCLAHE(clipLimit=127.0, tileGridSize=(8,8))
            clahe_img = clahe.apply(image)
            
            # Thresholding + Otsu
            otsu = cv.threshold(clahe_img, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
            
            # Erosion & Dilation
            kernel = np.ones((3,3), np.uint8)
            erosion = cv.erode(otsu, kernel,iterations=1)
            dilation = cv.dilate(erosion, kernel,iterations=1)
            
            # Laplacian (feature detection)
            laplacian = cv.Laplacian(dilation, cv.CV_64F, ksize=3)
            #laplacian = cv.Laplacian(otsu, cv.CV_8U, ksize=3)
            
            image= cv.resize(laplacian, (IMG_SIZE, IMG_SIZE), interpolation = cv.INTER_AREA)
            
            image_data.append(image)
            label_data.append(class_name)
            
    image_data = np.array(image_data)
    label_data = np.array(label_data)
    return image_data, label_data

image_data, label_data = create_dataset('/content/training_data_64')
print('The number of images data : {}'.format(image_data.shape))
print('The number of class data : {}'.format(label_data.shape))

The number of images data : (600, 64, 64)
The number of class data : (600,)


### PHOG

In [ ]:
# HOG
def hog_descriptor(images): 
    hog_fd = []
    hog_img = []
    for sample in images:
        fd, img = hog(sample, orientations=9,
                      pixels_per_cell=(8, 8), cells_per_block=(2,2), 
                      visualize=True, multichannel=False, feature_vector=True)

        hog_fd.append(fd)
        hog_img.append(img)

    hog_array = np.array(hog_fd, np.float32)
    vis_array = np.array(hog_img)
    return hog_array, vis_array


In [ ]:
##### feature descriptor using phog level 1
def phog_descriptor(images):
    hog_fd = []
    for sample in images:
        ### Level 0
        hog_vector0,_ = hog_descriptor([sample])
        level_0 = np.array(hog_vector0).ravel()
        
        ### Level 1
        # Cut the image
        cut = images.shape[1] // 2
        left = sample[:, :cut]
        right = sample[:, cut:]

        img1 = left[:cut, :]
        img2 = right[:cut, :]
        img3 = left[cut:, :]
        img4 = right[cut:, :]
        
        # hog
        sub_img = [img1, img2, img3, img4]
        hog_vector1,_ = hog_descriptor(sub_img)    # (4,324)
        level_1 = np.array(hog_vector1).ravel()
        
        ### Concatenate grid_1 and grid_2
        a1 = len(level_0)    # (1764)
        a2 = len(level_1)    # (1296)
        level_0_1 = []
        for i in range(max(a1, a2)):
            if i < a1:
                level_0_1.append(level_0[i])
            if i < a2:
                level_0_1.append(level_1[i])
        
        hog_fd.append(level_0_1)
        
    hog_fd = np.array(hog_fd)
    return hog_fd

hog_fd = phog_descriptor(image_data)

In [ ]:
##### feature descriptor using phog level 2
def phog_descriptor(images):
    hog_fd = []
    for sample in images:
        ### Level 0
        hog_vector0,_ = hog_descriptor([sample])    # (1, 1764)
        level_0 = np.array(hog_vector0).ravel()    # (1764)
        

        ### Level 1
        # Cut the image
        cut = images.shape[1] // 2
        left = sample[:, :cut]
        right = sample[:, cut:]

        img1 = left[:cut, :]
        img2 = right[:cut, :]
        img3 = left[cut:, :]
        img4 = right[cut:, :]
        
        sub_img_4 = [img1, img2, img3, img4]    # (4, 32,32)
        sub_img_4 = np.array(sub_img_4)
        hog_vector1,_ = hog_descriptor(sub_img_4)    # (4, 324)
        level_1 = np.array(hog_vector1).ravel()    # # (1296)
            

        ### Level 2
        sub_img_16 = []
        cut = images.shape[1] // 4
        img_row1 = sample[:cut, :]
        img_row2 = sample[cut:cut*2, :]
        img_row3 = sample[cut*2:cut*3, :]
        img_row4 = sample[cut*3:cut*4, :]

        sub_img_row = [img_row1, img_row2, img_row3, img_row4]
        #sub_img_row = np.array(sub_img_row)
        for img_row in sub_img_row:
            img_col1 = img_row[:, :cut]
            img_col2 = img_row[:, cut:cut*2]
            img_col3 = img_row[:, cut*2:cut*3]
            img_col4 = img_row[:, cut*3:cut*4]
            
            sub_img = [img_col1, img_col2, img_col3, img_col4]
            for img in sub_img:
                sub_img_16.append(img)
    
        sub_img_16 = np.array(sub_img_16)
        hog_vector2,_ = hog_descriptor(sub_img_16)    # (16, 36)
        level_2 = np.array(hog_vector2).ravel()    # (576)


        ### Concatenate grid_1 and grid_2
        a1 = len(level_0)    # (1764)
        a2 = len(level_1)    # (1296)
        a3 = len(level_2)    # (576)
        level_0_1_2 = []
        for i in range(max(a1, a2, a3)):
            if i < a1:
                level_0_1_2.append(level_0[i])
            if i < a2:
                level_0_1_2.append(level_1[i])
            if i < a3:
                level_0_1_2.append(level_2[i])
                
        hog_fd.append(level_0_1_2)
    hog_fd = np.array(hog_fd)    # (300 x 3636)
    print(hog_fd.shape)
    
    return hog_fd

hog_fd = phog_descriptor(image_data)

(600, 3636)


In [ ]:
##### feature descriptor using phog level 3
def phog_descriptor(images):
    hog_fd = []
    for sample in images:
        ### Level 0
        hog_vector0,_ = hog_descriptor([sample])    # (1, 1764)
        level_0 = np.array(hog_vector0).ravel()    # (1764)
        

        ### Level 1
        # Cut the image
        cut = images.shape[1] // 2
        left = sample[:, :cut]
        right = sample[:, cut:]

        img1 = left[:cut, :]
        img2 = right[:cut, :]
        img3 = left[cut:, :]
        img4 = right[cut:, :]
        
        sub_img_4 = [img1, img2, img3, img4]    # (4, 32,32)
        sub_img_4 = np.array(sub_img_4)
        hog_vector1,_ = hog_descriptor(sub_img_4)    # (4, 324)
        level_1 = np.array(hog_vector1).ravel()    # # (1296)
            

        ### Level 2
        sub_img_16 = []
        cut = images.shape[1] // 4
        img_row1 = sample[:cut, :]
        img_row2 = sample[cut:cut*2, :]
        img_row3 = sample[cut*2:cut*3, :]
        img_row4 = sample[cut*3:cut*4, :]

        sub_img_row = [img_row1, img_row2, img_row3, img_row4]
        #sub_img_row = np.array(sub_img_row)
        for img_row in sub_img_row:
            img_col1 = img_row[:, :cut]
            img_col2 = img_row[:, cut:cut*2]
            img_col3 = img_row[:, cut*2:cut*3]
            img_col4 = img_row[:, cut*3:cut*4]
            
            sub_img = [img_col1, img_col2, img_col3, img_col4]
            for img in sub_img:
                sub_img_16.append(img)
    
        sub_img_16 = np.array(sub_img_16)
        hog_vector2,_ = hog_descriptor(sub_img_16)    # (16, 36)
        level_2 = np.array(hog_vector2).ravel()    # (576)


        ### Level 3
        sub_img_64 = []
        cut = images.shape[1] // 8
        img_row1 = sample[:cut, :]
        img_row2 = sample[cut:cut*2, :]
        img_row3 = sample[cut*2:cut*3, :]
        img_row4 = sample[cut*3:cut*4, :]
        img_row5 = sample[cut*4:cut*5, :]
        img_row6 = sample[cut*5:cut*6, :]
        img_row7 = sample[cut*6:cut*7, :]
        img_row8 = sample[cut*7:, :]

        sub_img_row = [img_row1, img_row2, img_row3, img_row4, 
                    img_row5, img_row6, img_row7, img_row8]

        for img_row in sub_img_row:
            img_col1 = img_row[:, :cut]
            img_col2 = img_row[:, cut:cut*2]
            img_col3 = img_row[:, cut*2:cut*3]
            img_col4 = img_row[:, cut*3:cut*4]
            img_col5 = img_row[:, cut*4:cut*5]
            img_col6 = img_row[:, cut*5:cut*6]
            img_col7 = img_row[:, cut*6:cut*7]
            img_col8 = img_row[:, cut*7:]
            
            sub_img = [img_col1, img_col2, img_col3, img_col4, 
                    img_col5, img_col6, img_col7, img_col8]
            for img in sub_img:
                sub_img_64.append(img)
    
        sub_img_64 = np.array(sub_img_64)
        hog_vector3,_ = hog_descriptor(sub_img_64)
        level_3 = np.array(hog_vector3).ravel()


        ### Concatenate grid_1 and grid_2
        a1 = len(level_0)    # (1764)
        a2 = len(level_1)    # (1296)
        a3 = len(level_2)    # (576)
        a4 = len(level_3)
        level_0_1_2_3 = []
        for i in range(max(a1, a2, a3)):
            if i < a1:
                level_0_1_2_3.append(level_0[i])
            if i < a2:
                level_0_1_2_3.append(level_1[i])
            if i < a3:
                level_0_1_2_3.append(level_2[i])
            if i < a4:
                level_0_1_2_3.append(level_3[i])
                
        hog_fd.append(level_0_1_2_3)
    hog_fd = np.array(hog_fd)    # (300 x 3636)
    print(hog_fd.shape)
    
    return hog_fd

hog_fd = phog_descriptor(sample_data)

(1, 3636)
